# House Prices: Advanced Regression Techniques

In [292]:

import pandas as pd
import numpy as nop
import tensorflow as tf
from datetime import datetime

In [293]:
train = pd.read_csv('./data/train.csv')

In [294]:
test = pd.read_csv('./data/test.csv')

In [295]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [296]:
train.shape

(1460, 81)

In [297]:
train.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 81, dtype: object

In [298]:
# Attributes which are mostly Null
mostly_nan = {x:y for x,y in dict(train.isnull().sum()).items() if y > 700}
mostly_nan

{'Alley': 1369, 'PoolQC': 1453, 'Fence': 1179, 'MiscFeature': 1406}

In [299]:
train = train.drop(mostly_nan.keys(), axis = 1)

In [300]:
have_nan = {x:y for x,y in dict(train.isnull().sum()).items() if y > 0}
have_nan

{'LotFrontage': 259,
 'MasVnrType': 8,
 'MasVnrArea': 8,
 'BsmtQual': 37,
 'BsmtCond': 37,
 'BsmtExposure': 38,
 'BsmtFinType1': 37,
 'BsmtFinType2': 38,
 'Electrical': 1,
 'FireplaceQu': 690,
 'GarageType': 81,
 'GarageYrBlt': 81,
 'GarageFinish': 81,
 'GarageQual': 81,
 'GarageCond': 81}

In [301]:
train['Electrical'].value_counts().index[0]

'SBrkr'

In [302]:
train[train['Electrical'].isnull()]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1379,1380,80,RL,73.0,9735,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,5,2008,WD,Normal,167500


In [303]:
# Replace Nan with mode
for col,_ in have_nan.items():
    if train.dtypes[col] == 'object':
        mode = train[col].value_counts().index[0]
        train[col] = train[col].replace(np.nan, mode)
    else:
        median = train[col].median()
        train[col] = train[col].replace(np.nan, median)

In [304]:
{x:y for x,y in dict(train.isnull().sum()).items() if y > 0}

{}

In [305]:
categoricals = list(train.select_dtypes(include=['object']).columns)

In [306]:
from sklearn.preprocessing import LabelBinarizer
encoders = {}
to_drop = []
for col in categoricals:
    print(f'{col} = {train[col].unique()}')
    lb = LabelBinarizer()
    encoded = lb.fit_transform(train[col])
#     print(y.shape)
    if encoded.shape[1] > 1:
        ne = pd.DataFrame(encoded, columns=[f'{col}_{clas}' for clas in lb.classes_])
        to_drop.append(col)
    else:
        ne = pd.DataFrame(encoded, columns=[col])
    train = train.drop([col], axis = 1)
    train = pd.concat([train, ne], axis=1)
    encoders[col] = lb

MSZoning = ['RL' 'RM' 'C (all)' 'FV' 'RH']
Street = ['Pave' 'Grvl']
LotShape = ['Reg' 'IR1' 'IR2' 'IR3']
LandContour = ['Lvl' 'Bnk' 'Low' 'HLS']
Utilities = ['AllPub' 'NoSeWa']
LotConfig = ['Inside' 'FR2' 'Corner' 'CulDSac' 'FR3']
LandSlope = ['Gtl' 'Mod' 'Sev']
Neighborhood = ['CollgCr' 'Veenker' 'Crawfor' 'NoRidge' 'Mitchel' 'Somerst' 'NWAmes'
 'OldTown' 'BrkSide' 'Sawyer' 'NridgHt' 'NAmes' 'SawyerW' 'IDOTRR'
 'MeadowV' 'Edwards' 'Timber' 'Gilbert' 'StoneBr' 'ClearCr' 'NPkVill'
 'Blmngtn' 'BrDale' 'SWISU' 'Blueste']
Condition1 = ['Norm' 'Feedr' 'PosN' 'Artery' 'RRAe' 'RRNn' 'RRAn' 'PosA' 'RRNe']
Condition2 = ['Norm' 'Artery' 'RRNn' 'Feedr' 'PosN' 'PosA' 'RRAn' 'RRAe']
BldgType = ['1Fam' '2fmCon' 'Duplex' 'TwnhsE' 'Twnhs']
HouseStyle = ['2Story' '1Story' '1.5Fin' '1.5Unf' 'SFoyer' 'SLvl' '2.5Unf' '2.5Fin']
RoofStyle = ['Gable' 'Hip' 'Gambrel' 'Mansard' 'Flat' 'Shed']
RoofMatl = ['CompShg' 'WdShngl' 'Metal' 'WdShake' 'Membran' 'Tar&Grv' 'Roll'
 'ClyTile']
Exterior1st = ['VinylSd' 'Meta

In [307]:
train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,0,1,0,0,0,0,1,0


In [308]:
train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,0,0,1,0,0,0,0,1,0
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,0,0,0,1,0,0,0,0,1,0
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,0,0,1,0,0,0,0,1,0
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,0,0,1,1,0,0,0,0,0
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,0,0,0,1,0,0,0,0,1,0


In [309]:
train.dtypes.value_counts()

int32      236
int64       35
float64      3
dtype: int64

In [310]:
y = train['SalePrice']
X = train
X = train.drop(['SalePrice', 'Id'], axis = 1)

In [311]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
scalers = {}
for col in X:
    scale = StandardScaler()
    X[col] = scale.fit_transform(np.array(X[col]).reshape(-1, 1))
    scalers[col] = scale


In [147]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [148]:
from sklearn.linear_model import LinearRegression

In [149]:
lr_model = LinearRegression()

In [150]:
lr_model.fit(X_train, y_train)

LinearRegression()

In [151]:
pred = lr_model.predict(X_test)

In [152]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [153]:
mean_squared_error(y_test, pred)

1.329961960191654e+29

In [154]:
mean_absolute_error(y_test, pred)

50317047556976.33

In [312]:
test = test.drop(mostly_nan, axis = 1)

In [313]:
test_have_nan = {x:y for x,y in dict(test.isnull().sum()).items() if y > 0}
test_have_nan

{'MSZoning': 4,
 'LotFrontage': 227,
 'Utilities': 2,
 'Exterior1st': 1,
 'Exterior2nd': 1,
 'MasVnrType': 16,
 'MasVnrArea': 15,
 'BsmtQual': 44,
 'BsmtCond': 45,
 'BsmtExposure': 44,
 'BsmtFinType1': 42,
 'BsmtFinSF1': 1,
 'BsmtFinType2': 42,
 'BsmtFinSF2': 1,
 'BsmtUnfSF': 1,
 'TotalBsmtSF': 1,
 'BsmtFullBath': 2,
 'BsmtHalfBath': 2,
 'KitchenQual': 1,
 'Functional': 2,
 'FireplaceQu': 730,
 'GarageType': 76,
 'GarageYrBlt': 78,
 'GarageFinish': 78,
 'GarageCars': 1,
 'GarageArea': 1,
 'GarageQual': 78,
 'GarageCond': 78,
 'SaleType': 1}

In [314]:
# Replace Nan with mode
for col,_ in test_have_nan.items():
    mode = test[col].value_counts().index[0]
    test[col] = test[col].replace(np.nan, mode)

In [315]:
for col in categoricals:
#     print(f'{col} = {test[col].unique()}')

    encoded = encoders[col].transform(test[col])
#     print(y.shape)
    if encoded.shape[1] > 1:
        ne = pd.DataFrame(encoded, columns=[f'{col}_{clas}' for clas in encoders[col].classes_])
        to_drop.append(col)
    else:
        ne = pd.DataFrame(encoded, columns=[col])
    test = test.drop([col], axis = 1)
    test = pd.concat([test, ne], axis=1)

In [316]:
test.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,0,0,0,1,0,0,0,0,1,0
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,0,0,0,1,0,0,0,0,1,0
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,0,0,0,1,0,0,0,0,1,0
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,...,0,0,0,1,0,0,0,0,1,0
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,...,0,0,0,1,0,0,0,0,1,0


In [317]:
Id_col = test['Id']
X_test = test.drop(['Id'], axis = 1)

In [318]:
scalers

{'MSSubClass': StandardScaler(),
 'LotFrontage': StandardScaler(),
 'LotArea': StandardScaler(),
 'OverallQual': StandardScaler(),
 'OverallCond': StandardScaler(),
 'YearBuilt': StandardScaler(),
 'YearRemodAdd': StandardScaler(),
 'MasVnrArea': StandardScaler(),
 'BsmtFinSF1': StandardScaler(),
 'BsmtFinSF2': StandardScaler(),
 'BsmtUnfSF': StandardScaler(),
 'TotalBsmtSF': StandardScaler(),
 '1stFlrSF': StandardScaler(),
 '2ndFlrSF': StandardScaler(),
 'LowQualFinSF': StandardScaler(),
 'GrLivArea': StandardScaler(),
 'BsmtFullBath': StandardScaler(),
 'BsmtHalfBath': StandardScaler(),
 'FullBath': StandardScaler(),
 'HalfBath': StandardScaler(),
 'BedroomAbvGr': StandardScaler(),
 'KitchenAbvGr': StandardScaler(),
 'TotRmsAbvGrd': StandardScaler(),
 'Fireplaces': StandardScaler(),
 'GarageYrBlt': StandardScaler(),
 'GarageCars': StandardScaler(),
 'GarageArea': StandardScaler(),
 'WoodDeckSF': StandardScaler(),
 'OpenPorchSF': StandardScaler(),
 'EnclosedPorch': StandardScaler(),
 

In [319]:
from sklearn.preprocessing import MinMaxScaler
for col in X_test:
    X_test[col] = scalers[col].transform(np.array(X_test[col]).reshape(-1, 1))
     


In [580]:
my_pred = lr_model.predict(X_test)

In [581]:
my_pred

array([109392.66174316, 152254.56469727, 186848.2947998 , ...,
       180758.60296631, 105995.24700928, 224207.48864746])

In [582]:
pred_series = pd.Series(my_pred, name='SalePrice')

In [583]:
output = pd.concat([Id_col, pred_series], axis=1)

In [584]:
output.head()

,Id,SalePrice
0,1461,109392.661743
1,1462,152254.564697
2,1463,186848.294800
3,1464,192210.252869
4,1465,205223.612793


In [592]:
output.to_csv('./submissions/lr_kaggle_submssion.csv', index = False)

In [593]:
# Simple Linear Regression got score 0.85796 and 4989 position

In [594]:
from sklearn.ensemble import RandomForestRegressor

In [595]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [596]:
rf_model = RandomForestRegressor()

In [597]:
rf_model.fit(X, y)

RandomForestRegressor()

In [598]:
Id_col = test['Id']
X_kaggle_test = test.drop(['Id'], axis = 1)

In [599]:
my_pred = rf_model.predict(X_kaggle_test)
pred_series = pd.Series(my_pred, name='SalePrice')
output = pd.concat([Id_col, pred_series], axis=1)
output.to_csv('rf_kaggle_submssion.csv', index = False)

In [600]:
# Random Forest Regressor got score 0.14668 and 2965 position

In [601]:
from sklearn.ensemble import GradientBoostingRegressor

In [602]:
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=0, loss='ls')

In [603]:
gb_model.fit(X, y)

GradientBoostingRegressor(max_depth=1, random_state=0)

In [604]:
my_pred = gb_model.predict(X_kaggle_test)
pred_series = pd.Series(my_pred, name='SalePrice')
output = pd.concat([Id_col, pred_series], axis=1)
output.to_csv('./submissions/gb_kaggle_submssion.csv', index = False)

In [605]:
 # GradientBoosting Regressor got score 0.17547

In [606]:
from sklearn.neural_network import MLPRegressor

In [607]:
mlp_model = MLPRegressor(random_state=1, max_iter=5000)

In [608]:
mlp_model.fit(X, y)

MLPRegressor(max_iter=5000, random_state=1)

In [609]:
my_pred = mlp_model.predict(X_kaggle_test)
pred_series = pd.Series(my_pred, name='SalePrice')
output = pd.concat([Id_col, pred_series], axis=1)
output.to_csv('./submissions/mlp_kaggle_submssion.csv', index = False)

In [610]:
#!pip3 install xgboost

In [226]:
import xgboost as xgb

In [227]:
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=11196)

In [228]:
xgb_model.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             random_state=11196, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             subsample=1, tree_method='exact', validate_parameters=1,
             verbosity=None)

In [229]:
my_pred = xgb_model.predict(X_test)
pred_series = pd.Series(my_pred, name='SalePrice')
output = pd.concat([Id_col, pred_series], axis=1)
output.to_csv(f'./submissions/xgb_kaggle_submssion_{datetime.now().strftime("%d-%b-%Y_(%H-%M-%S)")}.csv', index = False)

In [399]:
# XGB Regressor got score 0.14274 and 2645 position

In [234]:
!pip install mlxtend

In [235]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor
# from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from mlxtend.regressor import StackingCVRegressor
from sklearn.model_selection import KFold, cross_val_score


In [236]:
lasso = Lasso(alpha = 0.0005)
ridge = Ridge(alpha = 11.9)
elasticnet = ElasticNet(alpha = 0.001)
gbr = GradientBoostingRegressor(learning_rate = 0.1, loss = 'huber', max_depth = 1, max_features ='sqrt',
                                min_samples_leaf = 15, min_samples_split = 10, n_estimators =3000)
# lightgbm = LGBMRegressor(learning_rate = 0.01, n_estimators = 5000, num_leaves = 4)
xgboost = XGBRegressor(learning_rate = 0.01, max_depth = 3, n_estimators = 3500)

In [237]:
scvr = StackingCVRegressor(regressors=(ridge, lasso, elasticnet, gbr, xgboost),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)

In [278]:
scvr.fit(X, y)

c:\users\hassa\.virtualenvs\mit_deep_learning-6vn7g6jr\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 114562553810.28876, tolerance: 749680604.11125
  positive)
c:\users\hassa\.virtualenvs\mit_deep_learning-6vn7g6jr\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 113478414962.83917, tolerance: 653204084.4196465
  positive)
c:\users\hassa\.virtualenvs\mit_deep_learning-6vn7g6jr\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:531: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 156706291684.53162, tolerance: 751884521.6800628
  positive)
c:\users\hassa\.virtualenvs\mit_deep_learning-6vn7g6jr\lib\site-packages\sklearn\linear_model\_coordinate_desce

StackingCVRegressor(meta_regressor=XGBRegressor(base_score=None, booster=None,
                                                colsample_bylevel=None,
                                                colsample_bynode=None,
                                                colsample_bytree=None,
                                                gamma=None, gpu_id=None,
                                                importance_type='gain',
                                                interaction_constraints=None,
                                                learning_rate=0.01,
                                                max_delta_step=None,
                                                max_depth=3,
                                                min_child_weight=None,
                                                missing=nan,
                                                monotone_constraints=None,
                                                n_estimators=3500, n_...
      

In [279]:
my_pred = scvr.predict(X_test)
pred_series = pd.Series(my_pred, name='SalePrice')
output = pd.concat([Id_col, pred_series], axis=1)
output.to_csv(f'./submissions/scvr_kaggle_submssion_{datetime.now().strftime("%d-%b-%Y_(%H-%M-%S)")}.csv', index = False)

ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', '

In [37]:
!pip install git+https://github.com/tensorflow/docs

  Cloning https://github.com/tensorflow/docs to c:\users\hassa\appdata\local\temp\pip-req-build-f9yi3uxp
  Created wheel for tensorflow-docs: filename=tensorflow_docs-0.0.0ddfe55f6903a44ad2e7a214f7578f6d349ef4a8a_-py3-none-any.whl size=111063 sha256=b03f318820f7cc78cec6ff8cab78c4d5beba0c0311ecc3b91693697fad7e372c
  Stored in directory: C:\Users\hassa\AppData\Local\Temp\pip-ephem-wheel-cache-83xa9v7f\wheels\cc\c4\d8\5341e93b6376c5c929c49469fce21155eb69cef1a4da4ce32c
Successfully built tensorflow-docs


  Running command git clone -q https://github.com/tensorflow/docs 'C:\Users\hassa\AppData\Local\Temp\pip-req-build-f9yi3uxp'


In [91]:
%load_ext tensorboard

In [183]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [192]:
model = Sequential()

In [ ]:
dense_layer = Dense(5)

In [322]:
def build_model():
  model = Sequential([
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1)
  ])

  optimizer = tf.keras.optimizers.Adam()

  model.compile(loss='msle',
                optimizer=optimizer,
                metrics=['mae', 'mse', 'msle'])
  return model

In [323]:
x_input = tf.constant([[1,2.]], shape=(1,2))

In [324]:
model = build_model()

In [325]:
EPOCHS = 1000
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
history = model.fit(
  X, y,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)])


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
# %tensorboard --logdir logs/fit

In [326]:
pred = model.predict(X_test).flatten()

In [327]:
pred

array([145048.55 , 166108.72 , 190419.4  , ..., 179819.02 , 110505.414,
       240974.47 ], dtype=float32)

In [328]:
pred_series = pd.Series(pred, name='SalePrice')
output = pd.concat([Id_col, pred_series], axis=1)
output.to_csv(f'./submissions/tf_kaggle_submssion_{datetime.now().strftime("%d-%b-%Y_(%H-%M-%S)")}.csv', index = False)